# Time-Frequency Consistency (TFC) + CNN PFF

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
from datetime import datetime
import lightning as L
import torch
import torchmetrics
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.loggers import CSVLogger

from minerva.analysis.metrics.balanced_accuracy import BalancedAccuracy
from minerva.analysis.model_analysis import TSNEAnalysis
from minerva.data.data_modules.har import MultiModalHARSeriesDataModule
from minerva.data.data_modules.har_rodrigues_24 import HARDataModuleCPC
from minerva.models.loaders import FromPretrained
from minerva.models.nets.time_series.cnns import CNN_PF_Backbone
from minerva.models.ssl.tfc import TFC_Model, TFC_Backbone
from minerva.pipelines.lightning_pipeline import SimpleLightningPipeline

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
execution_id = f'run_{datetime.now().strftime("%Y%m%d-%H%M%S")}'
log_dir = f"./logs/{execution_id}" 

print(f"Execution ID: {execution_id}")
print(f"Log dir: {log_dir}")

Execution ID: run_20260127-022407
Log dir: ./logs/run_20260127-022407


## Pre-training with TFC

### 1. Defining the Data Module

In [4]:
data_module = HARDataModuleCPC(
    data_path="/home/gustavo-luz/code/hiaac/paper_access_private_git/Benchmarking_Enconders_SSL/shared_data/rodrigues_2024_datasets/1-1/wisdm",
    input_size=6,
    window=60,
    overlap=60,
    batch_size=64
)

data_module

HARDataModuleCPC(batch_size=64, datasets=/workspaces/HIAAC-KR-Dev-Container/shared_data/rodrigues_2024_datasets/1-1/wisdm)

### 2. Defining the TF-C Model

In [6]:
# Get a batch of data
from minerva.models.nets.tnc import TSEncoder
data_module.setup("fit")
dataset = data_module.train_dataloader()
batch_x, batch_y = next(iter(dataset))
from minerva.models.nets.tnc import RnnEncoder
# Create the model and forward the batch to check the output shape
g_enc = RnnEncoder(
        hidden_size=100,
        in_channel=6,
        encoding_size=320,
        bidirectional=True,
        num_layers=1,
        dropout=0,
        cell_type='GRU',
        # device='cuda',
        permute=True
    )
r = g_enc.forward(batch_x)
r.shape

torch.Size([64, 320])

In [8]:
backbone = TFC_Backbone(
    input_channels=6,
    TS_length=60,
    single_encoding_size=128,
    time_encoder=RnnEncoder(
        hidden_size=100,
        in_channel=6,
        encoding_size=320,
        bidirectional=True,
        num_layers=1,
        dropout=0,
        cell_type='GRU',
        # device='cuda',
        permute=True
    ),
    frequency_encoder=RnnEncoder(
        hidden_size=100,
        in_channel=6,
        encoding_size=320,
        bidirectional=True,
        num_layers=1,
        dropout=0,
        cell_type='GRU',
        # device='cuda',
        permute=True
    ),
)
model = TFC_Model(
    input_channels=6,
    batch_size=64,
    # TS_length=r.shape[-1],
    TS_length=60,
    num_classes=6,
    single_encoding_size=128,
    backbone=backbone,
    pred_head=None,
)

model

TFC_Model(
  (backbone): TFC_Backbone(
    (time_encoder): RnnEncoder(
      (nn): Sequential(
        (0): Linear(in_features=200, out_features=320, bias=True)
      )
      (rnn): GRU(6, 100, bidirectional=True)
    )
    (frequency_encoder): RnnEncoder(
      (nn): Sequential(
        (0): Linear(in_features=200, out_features=320, bias=True)
      )
      (rnn): GRU(6, 100, bidirectional=True)
    )
    (time_projector): TFC_Standard_Projector(
      (projector): Sequential(
        (0): Linear(in_features=320, out_features=256, bias=True)
        (1): IgnoreWhenBatch1(
          (module): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (2): ReLU()
        (3): Linear(in_features=256, out_features=128, bias=True)
      )
    )
    (frequency_projector): TFC_Standard_Projector(
      (projector): Sequential(
        (0): Linear(in_features=320, out_features=256, bias=True)
        (1): IgnoreWhenBatch1(
          (module): BatchNorm1

### 3. Defining the Pytorch Lightning Trainer Configuration

In [9]:
## Callbacks
checkpoint_callback = ModelCheckpoint(
    dirpath='checkpoints/',
    monitor='val_loss',
    mode='min',
    save_last=True
)

## Logger
logger = CSVLogger(save_dir=log_dir, name='tfc-cnn-pff-pretraining', version=execution_id)

## Trainer
trainer = L.Trainer(
    max_epochs=3,
    accelerator="gpu",
    devices=1,
    logger=logger,
    callbacks=[checkpoint_callback],
    # Only for testing. Remove for production. We will only train using 1 batch
    limit_train_batches=1,
    limit_val_batches=1,
)

trainer

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer(limit_train_batches=1)` was configured so 1 batch per epoch will be used.
`Trainer(limit_val_batches=1)` was configured so 1 batch will be used.


### 4. Creating the training pipeline (and running the training)

In [10]:
train_pipeline = SimpleLightningPipeline(
    model=model,
    trainer=trainer,
    log_dir=log_dir,
    save_run_status=True,
    seed=42
)
train_pipeline.run(data_module, task="fit")

/usr/local/lib/python3.10/dist-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory /workspaces/HIAAC-KR-Dev-Container/Minerva-Exps/benchmarks/experiments/docs/tfc/codecarbon/checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



  | Name     | Type            | Params | Mode 
-----------------------------------------------------
0 | backbone | TFC_Backbone    | 489 K  | train
1 | loss_fn  | NTXentLoss_poly | 0      | train
-----------------------------------------------------
489 K     Trainable params
0         Non-trainable params
489 K     Total params
1.958     Total estimated model params size (MB)
27        Modules in train mode
0         Modules in eval mode


** Seed set to: 42 **
Pipeline info saved at: /workspaces/HIAAC-KR-Dev-Container/Minerva-Exps/benchmarks/experiments/docs/tfc/codecarbon/logs/run_20260127-022407/run_2026-01-27-02-25-43f6a68df2.yaml


/usr/local/lib/python3.10/dist-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 2: 100%|██████████| 1/1 [00:00<00:00,  1.02it/s, v_num=2407, val_loss=4.770]

`Trainer.fit` stopped: `max_epochs=3` reached.


Epoch 2: 100%|██████████| 1/1 [00:01<00:00,  0.83it/s, v_num=2407, val_loss=4.770]
⏱️ fit took 5.06s → saved to /workspaces/HIAAC-KR-Dev-Container/Minerva-Exps/benchmarks/experiments/docs/tfc/codecarbon/logs/run_20260127-022407/timings_fit.csv
Pipeline info saved at: /workspaces/HIAAC-KR-Dev-Container/Minerva-Exps/benchmarks/experiments/docs/tfc/codecarbon/logs/run_20260127-022407/run_2026-01-27-02-25-43f6a68df2.yaml


### 5. Inspecting checkpoints

In [11]:
ckpt_path = checkpoint_callback.last_model_path
ckpt = torch.load(ckpt_path, map_location="cuda")
ckpt = ckpt.get("state_dict", ckpt)
list(ckpt.keys())

['backbone.time_encoder.nn.0.weight',
 'backbone.time_encoder.nn.0.bias',
 'backbone.time_encoder.rnn.weight_ih_l0',
 'backbone.time_encoder.rnn.weight_hh_l0',
 'backbone.time_encoder.rnn.bias_ih_l0',
 'backbone.time_encoder.rnn.bias_hh_l0',
 'backbone.time_encoder.rnn.weight_ih_l0_reverse',
 'backbone.time_encoder.rnn.weight_hh_l0_reverse',
 'backbone.time_encoder.rnn.bias_ih_l0_reverse',
 'backbone.time_encoder.rnn.bias_hh_l0_reverse',
 'backbone.frequency_encoder.nn.0.weight',
 'backbone.frequency_encoder.nn.0.bias',
 'backbone.frequency_encoder.rnn.weight_ih_l0',
 'backbone.frequency_encoder.rnn.weight_hh_l0',
 'backbone.frequency_encoder.rnn.bias_ih_l0',
 'backbone.frequency_encoder.rnn.bias_hh_l0',
 'backbone.frequency_encoder.rnn.weight_ih_l0_reverse',
 'backbone.frequency_encoder.rnn.weight_hh_l0_reverse',
 'backbone.frequency_encoder.rnn.bias_ih_l0_reverse',
 'backbone.frequency_encoder.rnn.bias_hh_l0_reverse',
 'backbone.time_projector.projector.0.weight',
 'backbone.time_pro

## Fine-tuning with CNN PFF

### 1. Defining the Data Module

In [12]:
data_module = MultiModalHARSeriesDataModule(
    data_path="/home/gustavo-luz/code/hiaac/paper_access_private_git/Benchmarking_Enconders_SSL/shared_data/daghar/standardized_view/MotionSense/",
    feature_prefixes=["accel-x", "accel-y", "accel-z", "gyro-x", "gyro-y", "gyro-z"],
    label="standard activity code",
    features_as_channels=True,
    cast_to="float32",
    batch_size=64,
)

data_module

MultiModalHARSeriesDataModule(data_path=/workspaces/HIAAC-KR-Dev-Container/shared_data/daghar/standardized_view/MotionSense, batch_size=64)

In [13]:
# Pega os dataloaders de treino e validação
data_module.setup("fit")
train_data_loader = data_module.train_dataloader()
validation_data_loader = data_module.val_dataloader()
first_batch = next(iter(train_data_loader))

X, y = first_batch
print(X.shape, y.shape)

Using DataLoader with shuffle=True
Using DataLoader with shuffle=False
torch.Size([64, 6, 60]) torch.Size([64])


### 2. Defining the CNN PFF Model for Fine-tuning

In [15]:
backbone = TFC_Backbone(
    input_channels=6,
    TS_length=60,
    single_encoding_size=128,
    time_encoder=RnnEncoder(
        hidden_size=100,
        in_channel=6,
        encoding_size=320,
        bidirectional=True,
        num_layers=1,
        dropout=0,
        cell_type='GRU',
        # device='cuda',
        permute=True
    ),
    frequency_encoder=RnnEncoder(
        hidden_size=100,
        in_channel=6,
        encoding_size=320,
        bidirectional=True,
        num_layers=1,
        dropout=0,
        cell_type='GRU',
        # device='cuda',
        permute=True
    ),
)


backbone = FromPretrained(
    model=backbone, 
    ckpt_path=checkpoint_callback.best_model_path,
    # filter_keys=["backbone"],
    keys_to_rename={"backbone.": ""},      # Let's remove the prefix from the keys
                                        # on the checkpoint to load the model
                                        # correctly
    strict=True,
    error_on_missing_keys=True
)

Performing key renaming with: {'backbone.': ''}
	Renaming key: backbone.time_encoder.nn.0.weight -> time_encoder.nn.0.weight (changed: True)
	Renaming key: backbone.time_encoder.nn.0.bias -> time_encoder.nn.0.bias (changed: True)
	Renaming key: backbone.time_encoder.rnn.weight_ih_l0 -> time_encoder.rnn.weight_ih_l0 (changed: True)
	Renaming key: backbone.time_encoder.rnn.weight_hh_l0 -> time_encoder.rnn.weight_hh_l0 (changed: True)
	Renaming key: backbone.time_encoder.rnn.bias_ih_l0 -> time_encoder.rnn.bias_ih_l0 (changed: True)
	Renaming key: backbone.time_encoder.rnn.bias_hh_l0 -> time_encoder.rnn.bias_hh_l0 (changed: True)
	Renaming key: backbone.time_encoder.rnn.weight_ih_l0_reverse -> time_encoder.rnn.weight_ih_l0_reverse (changed: True)
	Renaming key: backbone.time_encoder.rnn.weight_hh_l0_reverse -> time_encoder.rnn.weight_hh_l0_reverse (changed: True)
	Renaming key: backbone.time_encoder.rnn.bias_ih_l0_reverse -> time_encoder.rnn.bias_ih_l0_reverse (changed: True)
	Renaming key

In [16]:
from minerva.models.nets.mlp import MLP
head = MLP([256, 128, 6])
head

MLP(
  (0): Linear(in_features=256, out_features=128, bias=True)
  (1): ReLU()
  (2): Linear(in_features=128, out_features=6, bias=True)
)

In [17]:
from minerva.models.nets.base import SimpleSupervisedModel
model = SimpleSupervisedModel(
    backbone=backbone,
    fc=head,
    loss_fn=torch.nn.CrossEntropyLoss(),
    flatten=False,
    train_metrics = {"acc": torchmetrics.Accuracy(task="multiclass", num_classes=6)},
    val_metrics = {"acc": torchmetrics.Accuracy(task="multiclass", num_classes=6)},
    test_metrics = {"acc": torchmetrics.Accuracy(task="multiclass", num_classes=6)}
)

model

SimpleSupervisedModel(
  (backbone): TFC_Backbone(
    (time_encoder): RnnEncoder(
      (nn): Sequential(
        (0): Linear(in_features=200, out_features=320, bias=True)
      )
      (rnn): GRU(6, 100, bidirectional=True)
    )
    (frequency_encoder): RnnEncoder(
      (nn): Sequential(
        (0): Linear(in_features=200, out_features=320, bias=True)
      )
      (rnn): GRU(6, 100, bidirectional=True)
    )
    (time_projector): TFC_Standard_Projector(
      (projector): Sequential(
        (0): Linear(in_features=320, out_features=256, bias=True)
        (1): IgnoreWhenBatch1(
          (module): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (2): ReLU()
        (3): Linear(in_features=256, out_features=128, bias=True)
      )
    )
    (frequency_projector): TFC_Standard_Projector(
      (projector): Sequential(
        (0): Linear(in_features=320, out_features=256, bias=True)
        (1): IgnoreWhenBatch1(
          (module)

In [18]:
# Option 1: Measure ALL model parameters (including backbone AND head)
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Total model parameters: {total_params:,}")
print(f"Trainable model parameters: {trainable_params:,}")

# Option 2: Measure components separately (as you intended)
# Backbone parameters
backbone_total = sum(p.numel() for p in model.backbone.parameters())
backbone_trainable = sum(p.numel() for p in model.backbone.parameters() if p.requires_grad)

# Head parameters (use model.fc, not head variable)
head_total = sum(p.numel() for p in model.fc.parameters())
head_trainable = sum(p.numel() for p in model.fc.parameters() if p.requires_grad)

print(f"\nBackbone:")
print(f"  Total parameters: {backbone_total:,}")
print(f"  Trainable parameters: {backbone_trainable:,}")

print(f"\nClassification Head:")
print(f"  Total parameters: {head_total:,}")
print(f"  Trainable parameters: {head_trainable:,}")

# Verify the sum matches Option 1
print(f"\nVerification:")
print(f"Backbone + Head total: {backbone_total + head_total:,}")
print(f"Should match total: {total_params:,}")

from thop import profile
evaluation_data = torch.rand(1000, 6, 60, device='cuda')
model.to('cuda')
macs, params = profile(model, inputs=(evaluation_data,))

print(f"MACs: {macs:,}")
print(f"Parameters: {params:,}")

# from codecarbon import EmissionsTracker

torch.cuda.is_available()
torch.cuda.device_count()

Total model parameters: 523,078
Trainable model parameters: 523,078

Backbone:
  Total parameters: 489,408
  Trainable parameters: 489,408

Classification Head:
  Total parameters: 33,670
  Trainable parameters: 33,670

Verification:
Backbone + Head total: 523,078
Should match total: 523,078
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_gru() for <class 'torch.nn.modules.rnn.GRU'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm1d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
MACs: 8,336,960,000.0
Parameters: 523,078.0


1

In [19]:
import pandas as pd
from codecarbon import EmissionsTracker

TOTAL_RUNS = 14
DISCARD_FIRST = 2
DISCARD_LAST = 2

results = []

tracker = EmissionsTracker(
    project_name="basic_measurement",
    measure_power_secs=10,
    save_to_file=False
)

try:
    for run_id in range(TOTAL_RUNS):
        tracker.start_task(f"measure_inference_{run_id}")

        _ = model(evaluation_data)  # inference

        emissions = tracker.stop_task()

        energy_kwh = emissions.energy_consumed
        energy_mwh = energy_kwh * 1_000          # kWh → mWh
        energy_j   = energy_kwh * 3_600_000      # kWh → J
        emissions_g = emissions.emissions * 1_000 * 1_000  # kg → g→ mg

        results.append({
            "run": run_id,
            "energy_mWh": energy_mwh,
            "energy_J": energy_j,
            "emissions_mgCO2eq": emissions_g,
            # "duration_s": emissions.duration
        })

finally:
    tracker.stop()

# --- All runs ---
df_all = pd.DataFrame(results)

# --- Valid runs (ignore first & last 2) ---
df_valid = df_all.iloc[DISCARD_FIRST: TOTAL_RUNS - DISCARD_LAST]

# --- Statistics ---
mean = df_valid[["energy_mWh", "energy_J", "emissions_mgCO2eq"]].mean()
std  = df_valid[["energy_mWh", "energy_J", "emissions_mgCO2eq"]].std()

summary = pd.DataFrame(
    [mean, std],
    index=["mean", "std"]
).round(2)

# --- Final table ---
df_final = pd.concat([
    df_valid.set_index("run").round(2),
    summary
])

df_final


[codecarbon WARNING @ 02:25:54] Multiple instances of codecarbon are allowed to run at the same time.
[codecarbon INFO @ 02:25:54] [setup] RAM Tracking...
[codecarbon INFO @ 02:25:54] [setup] CPU Tracking...
[codecarbon WARNING @ 02:25:55] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Linux OS detected: Please ensure RAPL files exist, and are readable, at /sys/class/powercap/intel-rapl/subsystem to measure CPU

[codecarbon INFO @ 02:25:55] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU E5-2630 v2 @ 2.60GHz
[codecarbon WARNING @ 02:25:55] No CPU tracking mode found. Falling back on CPU load mode.
[codecarbon INFO @ 02:25:55] [setup] GPU Tracking...
[codecarbon INFO @ 02:25:55] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 02:25:55] The below tracking methods have been set up:
                RAM Tracking Method: RAM power estimation model
                CPU Tracking Method: cpu_load
                GPU Tracking Method: pynvml
         

,energy_mWh,energy_J,emissions_mgCO2eq
2,0.01,44.67,1.22
3,0.01,45.65,1.25
4,0.01,45.29,1.24
5,0.01,45.08,1.23
6,0.01,40.43,1.10
7,0.01,44.90,1.23
8,0.01,45.07,1.23
9,0.01,40.05,1.09
10,0.01,45.16,1.23
11,0.01,45.50,1.24


In [20]:
a

NameError: name 'a' is not defined

### 3. Defining the Pytorch Lightning Trainer Configuration

In [ ]:
## Callbacks
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/", monitor="val_loss", mode="min", save_last=True
)

## Logger
logger = CSVLogger(
    save_dir=log_dir, name="tfc-cnn-pff-finetune", version=execution_id
)

## Trainer
trainer = L.Trainer(
    max_epochs=3,
    accelerator="gpu",
    devices=1,
    logger=logger,
    callbacks=[checkpoint_callback],
    # Only for testing. Remove for production. We will only train using 1 batch
    limit_train_batches=1,
    limit_val_batches=1,
)

trainer

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer(limit_train_batches=1)` was configured so 1 batch per epoch will be used.
`Trainer(limit_val_batches=1)` was configured so 1 batch will be used.


### 4. Creating the fine-tuning pipeline (and running the training)

In [ ]:
train_pipeline = SimpleLightningPipeline(
    model=model,
    trainer=trainer,
    log_dir=log_dir,
    save_run_status=True,
    seed=42
)
train_pipeline.run(data_module, task="fit")

Seed set to 42





 cuda device: 


 NVIDIA RTX A5000
Log directory set to: /workspaces/HIAAC-KR-Dev-Container/Minerva-Exps/benchmarks/experiments/docs/tfc/logs/run_20250224-204109
Pipeline info saved at: /workspaces/HIAAC-KR-Dev-Container/Minerva-Exps/benchmarks/experiments/docs/tfc/logs/run_20250224-204109/run_2025-02-24-20-41-222398384b394a48d39b8526e72946fcfd.yaml


/usr/local/lib/python3.10/dist-packages/lightning/pytorch/callbacks/model_checkpoint.py:652: Checkpoint directory /workspaces/HIAAC-KR-Dev-Container/Minerva-Exps/benchmarks/experiments/docs/tfc/checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [3]

  | Name     | Type             | Params | Mode 
------------------------------------------------------
0 | backbone | TFC_Backbone     | 489 K  | train
1 | fc       | MLP              | 33.7 K | train
2 | loss_fn  | CrossEntropyLoss | 0      | train
------------------------------------------------------
523 K     Trainable params
0         Non-trainable params
523 K     Total params
2.092     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]Using DataLoader with shuffle=False
Using DataLoader with shuffle=True                                         


/usr/local/lib/python3.10/dist-packages/lightning/pytorch/loops/fit_loop.py:298: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 2: 100%|██████████| 1/1 [00:06<00:00,  0.15it/s, v_num=4109, val_loss=1.770, val_acc=0.0312, train_loss=1.510, train_acc=0.391]

`Trainer.fit` stopped: `max_epochs=3` reached.


Epoch 2: 100%|██████████| 1/1 [00:06<00:00,  0.14it/s, v_num=4109, val_loss=1.770, val_acc=0.0312, train_loss=1.510, train_acc=0.391]
Pipeline info saved at: /workspaces/HIAAC-KR-Dev-Container/Minerva-Exps/benchmarks/experiments/docs/tfc/logs/run_20250224-204109/run_2025-02-24-20-41-222398384b394a48d39b8526e72946fcfd.yaml


## Evaluating the Fine-tuned Model

In [ ]:
test_pipeline = SimpleLightningPipeline(
    model=model,
    trainer=trainer,
    log_dir=log_dir,
    save_run_status=True,
    seed=42,
    classification_metrics={
        "accuracy": torchmetrics.Accuracy(num_classes=6, task="multiclass"),
        "f1": torchmetrics.F1Score(num_classes=6, task="multiclass"),
        "precision": torchmetrics.Precision(num_classes=6, task="multiclass"),
        "recall": torchmetrics.Recall(num_classes=6, task="multiclass"),
        "balanced_accuracy": BalancedAccuracy(num_classes=6, task="multiclass"),
    },
    apply_metrics_per_sample=False,
    # model_analysis={
    #     "tsne": TSNEAnalysis(
    #         height=800,
    #         width=800,
    #         legend_title="Activity",
    #         title="t-SNE of TFC Finetuned on MotionSense",
    #         output_filename="tsne_tfc_finetuned_motionsense.pdf",
    #         label_names={
    #             0: "sit",
    #             1: "stand",
    #             2: "walk",
    #             3: "stair up",
    #             4: "stair down",
    #             5: "run",
    #             6: "stair up and down",
    #         },
    #     )
    # },
)

test_pipeline.run(
    data_module, task="evaluate", ckpt_path=checkpoint_callback.best_model_path
)

Seed set to 42





 cuda device: 


 NVIDIA RTX A5000
Log directory set to: /workspaces/HIAAC-KR-Dev-Container/Minerva-Exps/benchmarks/experiments/docs/tfc/logs/run_20250224-204109
Pipeline info saved at: /workspaces/HIAAC-KR-Dev-Container/Minerva-Exps/benchmarks/experiments/docs/tfc/logs/run_20250224-204109/run_2025-02-24-20-41-4775648df007e74fdaa62111cd7339fbd9.yaml
Using DataLoader with shuffle=False


Restoring states from the checkpoint path at /workspaces/HIAAC-KR-Dev-Container/Minerva-Exps/benchmarks/experiments/docs/tfc/checkpoints/epoch=2-step=3-v17.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [3]
Loaded model weights from the checkpoint at /workspaces/HIAAC-KR-Dev-Container/Minerva-Exps/benchmarks/experiments/docs/tfc/checkpoints/epoch=2-step=3-v17.ckpt


Using DataLoader with shuffle=False
Predicting DataLoader 0: 100%|██████████| 17/17 [00:00<00:00, 88.13it/s]
Running classification metrics...
Metrics saved to /workspaces/HIAAC-KR-Dev-Container/Minerva-Exps/benchmarks/experiments/docs/tfc/logs/run_20250224-204109/metrics_2025-02-24-20-41-4775648df007e74fdaa62111cd7339fbd9.yaml
Pipeline info saved at: /workspaces/HIAAC-KR-Dev-Container/Minerva-Exps/benchmarks/experiments/docs/tfc/logs/run_20250224-204109/run_2025-02-24-20-41-4775648df007e74fdaa62111cd7339fbd9.yaml


/workspaces/HIAAC-KR-Dev-Container/Minerva-Dev/minerva/analysis/metrics/balanced_accuracy.py:59: UserWarning: y_pred contains nan values and not all classes passed
  warnings.warn(f"y_pred contains nan values and not all classes passed")


{'classification': {'accuracy': [0.4416195750236511],
  'f1': [0.4416195750236511],
  'precision': [0.4416195750236511],
  'recall': [0.4416195750236511],
  'balanced_accuracy': [0.4912859797477722]}}